# AiiDA-KKR demo

Here is a Demo to run the Voronoi code with a follow up KKR calculation with AiiDA 
with pure python code. Also add the end we run the same with the kkr_scf workchain

Some Comments:

In oder to run the KKR and Voronoi codes you have to set them up as codes in AiiDA.
You might source a bash rc in the pretext execution of the code for Licenzing issues.
Also you should symbol link the ElementPotential DataBase for the Voronoi code
    
If you want to test, use submit test, which will save all files created before a calculation would be run in a local test_submit folder

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import time
import os
from aiida import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()
from aiida.orm import Code, load_node
from aiida.orm import DataFactory, CalculationFactory
from aiida_kkr.tools.kkr_params import kkrparams
from pprint import pprint
from scipy import array
from aiida_kkr.calculations.kkr import KkrCalculation
from aiida_kkr.calculations.voro import VoronoiCalculation
from aiida_kkr.parsers.voro import VoronoiParser
from aiida_kkr.parsers.kkr import KkrParser

ParameterData = DataFactory('parameter')
StructureData = DataFactory('structure')

In [ ]:
# Prepare and fill AiiDA StructureData as input, example Cu

alat = 6.830000 # in a_Bohr
abohr = 0.52917721067
# number of atom positions in unit cell
natyp = 1
# bravais vectors
bravais = array([[0.5, 0.5, 0.0], [0.5, 0.0, 0.5], [0.0, 0.5, 0.5]])

a = 0.5*alat*abohr
Cu = StructureData(cell=[[a, a, 0.0], [a, 0.0, a], [0.0, a, a]])
Cu.append_atom(position=[0.0, 0.0, 0.0], symbols='Cu')
#Cu.store()
Cu = load_node(1)
print(Cu)

In [ ]:
# Now gernerate a ParameterData node with keyvalues needed by voronoi and KKR 
# we use a helper function for some defaults and set some values

keywords = kkrparams(NATYP=natyp, NSPIN=1, LMAX=2, RCLUSTZ=1.5, INS=0)
pprint(keywords.get_set_values())

In [ ]:
# Store the node
keyw = ParameterData(dict=keywords.get_dict())
#keyw.store()
keyw = load_node(2)
print keyw

In [ ]:
# Running a single Vornoi calculation
voronoi = Code.get_from_string('voronoi@iff003')
calc = VoronoiCalculation()
calc.label = 'Test voronoi'
calc.set_withmpi(False)
calc.set_resources({"num_machines" : 1})
calc.set_max_wallclock_seconds(300)
calc.set_computer(voronoi.get_computer())
calc.use_code(voronoi)
calc.use_structure(Cu)
calc.use_parameters(keyw)

In [ ]:
submit_test = False

if submit_test:
    subfolder, script_filename = calc.submit_test()
    print "Test_submit for calculation (uuid='{}')".format(
        calc.uuid)
    print "Submit file in {}".format(os.path.join(
        os.path.relpath(subfolder.abspath),
        script_filename
    ))
else:
    calc.store_all()
    print "created calculation; calc=Calculation(uuid='{}') # ID={}".format(
        calc.uuid, calc.dbnode.pk)
    calc.submit()
    print "submitted calculation; calc=Calculation(uuid='{}') # ID={}".format(
        calc.uuid, calc.dbnode.pk)

In [ ]:
!cat submit_test/20181120-00001/_aiidasubmit.sh
!ls submit_test/20181120-00001/
!cat submit_test/20181120-00001/inputcard

In [ ]:
# Ontop the voronoi calculation we want to run a KKR calculation
# for this we have to get some things from the voronoi calculation

In [ ]:
# use the calculation run before or load a voronoi calculation
calc2 = load_node(79565)

In [ ]:
# We create a new parameter node in which we store the emin extracted form the voronoi calculation
#emin = calc1.get_outputs_dict()['output_parameters'].get_dict()['EMIN']
emin = calc2.res.EMIN
remote = calc2.get_outputs_dict()['remote_folder']
keywords2 = keywords
keywords2['EMIN'][0] = emin


In [ ]:
keyw2 = ParameterData(dict=keywords2)
#keyw2.store()
keyw2 = load_node(79570)
print keyw2

In [ ]:
# Now we create and run the kkr Calculation
kkr = Code.get_from_string('kkr1@local_mac')#'kkrimp@local_mac')
calc1 = KkrCalculation()
calc1.label = 'Test kkr'
calc1.set_withmpi(False)
calc1.set_resources({"num_machines" : 1})
calc1.set_max_wallclock_seconds(300)
calc1.set_computer('local_mac')
calc1.use_code(kkr)
#calc1.use_structure(Cu)
calc1.use_parameters(keyw2)
calc1.use_parent_folder(remote)

In [ ]:
submit_test = False
if submit_test:
    subfolder, script_filename = calc1.submit_test()
    print "Test_submit for calculation (uuid='{}')".format(
        calc1.uuid)
    print "Submit file in {}".format(os.path.join(
        os.path.relpath(subfolder.abspath),
        script_filename
    ))
else:
    calc1.store_all()
    print "created calculation; calc=Calculation(uuid='{}') # ID={}".format(
        calc1.uuid, calc.dbnode.pk)
    calc1.submit()
    print "submitted calculation; calc=Calculation(uuid='{}') # ID={}".format(
        calc.uuid, calc.dbnode.pk)

In [ ]:
! cat submit_test/20171110-00020///_aiidasubmit.sh
! ls submit_test/20171110-00020//
! cat submit_test/20171110-00020/inputcard

In [ ]:
# Check with the verdi shell if everything with you calculations went right

In [ ]:
# Voronoi parser test

In [ ]:
n = load_node(79559)
retrieved_dict = {'retrieved' : n}
voro_parser = VoronoiParser(calc)
suc, nodes = voro_parser.parse_with_retrieved(retrieved_dict)
print suc
print nodes
print nodes[0][1].get_dict()['EMIN']

In [ ]:
# Test KKR parser functions

In [ ]:
from aiida_kkr.parsers.kkr import KkrParser, parse_kkr_outputfile

In [ ]:
outfile = '/Users/broeder/aiida/github/aiida-kkr/aiida_kkr/tests/files/kkr/outputfiles/out_kkr_Cu'

In [ ]:
res = parse_kkr_outputfile(outfile)

In [ ]:
print res

In [ ]:
# Test kkr_scf workchain

In [ ]:
from aiida_kkr.workflows.kkr_scf import kkr_scf_wc
from aiida.work import run, submit

In [ ]:
# use same nodes as above
Cu = load_node(79546)# structure
keyw = load_node(79550)# parameterdata keywords
wf_parameters = ParameterData(dict={})

#wf_parameters.store()
#wf_parameters = load_node()
print(wf_parameters)


In [ ]:
# The workflow is still very premitive, therefore we use for testing parameters that should work for both codes
res = run(kkr_scf_wc, structure=Cu, calc_parameters=keyw2, voronoi=voronoi, kkr=kkr)#wf_parameters=wf_parameters,

In [ ]:
print(res)

In [ ]:
res = submit(kkr_scf_wc, structure=Cu, calc_parameters=key2w, voronoi=voronoi, kkr=kkr)# wf_parameters=wf_parameters,